### +++ static +++

In [1]:
import opensim as osim
import os
from simFunctions import runProgram, scaleOptimalForceSubjectSpecific
# from stan_utils import subject_specific_isometric_force
import pandas as pd
os.getcwd()

'c:\\Users\\User\\OneDrive\\Documents\\WORK\\JRF_GaitAnalysis\\modelling_paper\\repo\\folder\\model_update'

# Udate Scaling XML

This file uses scaling scheme that is already given in scaling_setting.xml.

For walking, the model is scaled following distances between external markers and muscle forces are scaled by a factor calculated with mass and height of the subject relative the mass and the height of the generic model.

In [ ]:
# IMPORTANT : THESE SHOULD BE PERSON-SPECIFIC
mass_text = '52.4'
height_text = '1.60'
age_text = '33'

# path to experimental .trc file : <marker_file>
experimental_markers = r'../motion_lab/static/Static03/task.trc'
#r"C:\Users\User\OneDrive\Documents\WORK\JRF_GaitAnalysis\modelling_paper\repo\folder\motion_lab\static\Static03\task.trc"
static_df = pd.read_csv(experimental_markers, delimiter='\t', skiprows=3, header=[0,1], index_col=0)


In [25]:
# paths to and names of files

# <model_file>
path_to_generic_model = '../templates/RajagopalModified_generic_copy.osim'

# <marker_set_file>
path_to_generic_marker_set =  '../templates/generic_skin_markers.xml'# 'unassigned' '../templates/' generic_markers_with_joints.xml

# <time_range>'
time_range = f' 0 {list(static_df.loc[static_df.shape[0], "Time"])[0] }'

# <output_scale_file>
path_to_model = os.path.join("..", 'final_results', 'generic_scaled')
if not os.path.exists(path_to_model):
    print(path_to_model)
    os.makedirs(path_to_model)

output_scale_file = os.path.join(path_to_model, 'output_scale_file.txt')

output_model_file = os.path.join(path_to_model, f'scaled_model.osim')  # f'scaled_model_{ind}_joints.osim'

output_scaling_settings = os.path.join(path_to_model, f'scaling_setting.xml')

In [26]:
import xml.etree.ElementTree as ET

# parse the .xml with markers in body frames
scaling_tree=ET.parse("../templates/scaling_setting.xml")
scaling_root = scaling_tree.getroot()

for generic_model in scaling_root.iter('model_file'):
    generic_model.text = path_to_generic_model

for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_generic_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range

for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file

for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_generic_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range

for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file

for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

for marker_placer in scaling_root.iter('MarkerPlacer'):
    apply = marker_placer.find('apply')
    apply.text = 'true'

In [27]:
for mass in scaling_root.iter('mass'):
	mass.text = mass_text

for height in scaling_root.iter('height'):
	height.text = height_text

for age in scaling_root.iter('age'):
	age.text = age_text

In [28]:
scaling_tree.write(output_scaling_settings)

## 2.1 Scale model to markers

In [29]:
cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = output_scaling_settings
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

[info] Preparing to run ScaleTool.

[info] Processing subject Rajagopal-generic-scaled...

[info] Step 1: Loading generic model

[info] Loaded model RajagopalModifiedGeneric from file c:\Users\User\OneDrive\Documents\WORK\JRF_GaitAnalysis\modelling_paper\repo\folder\templates\RajagopalModified_generic_copy.osim

[warning] Couldn't find file 'r_pelvis.vtp'.

[warning] Couldn't find file 'l_pelvis.vtp'.

[warning] Couldn't find file 'sacrum.vtp'.

[warning] Couldn't find file 'r_femur.vtp'.

[warning] Couldn't find file 'r_tibia.vtp'.

[warning] Couldn't find file 'r_fibula.vtp'.

[warning] Couldn't find file 'r_patella.vtp'.

[warning] Couldn't find file 'r_talus.vtp'.

[warning] Couldn't find file 'r_foot.vtp'.

[warning] Couldn't find file 'r_bofoot.vtp'.

[warning] Couldn't find file 'l_femur.vtp'.

[warning] Couldn't find file 'l_tibia.vtp'.

[warning] Couldn't find file 'l_fibula.vtp'.

[warning] Couldn't find file 'l_patella.vtp'.

[warning] Couldn't find file 'l_talus.vtp'.

[war

## Scale model muscle forces to height and weight

In [30]:
generic_model = osim.Model(path_to_generic_model)
model=osim.Model(output_model_file)
scaled = scaleOptimalForceSubjectSpecific(generic_model, model, 1.60, float(height_text))


# Set <ignore_tendon_compliance> and <ignore_activation_dynamics> to true

In [31]:
muscle_set = scaled.getMuscles()
for i in range(muscle_set.getSize()):
    current = muscle_set.get(i)
    current.set_ignore_activation_dynamics(True)
    current.set_ignore_tendon_compliance(True)

## Increase Pelvis Tilt

In [11]:
# pelvis_coordinates = model.getCoordinateSet().get('pelvis_tilt')
# pelvis_coordinates.setRangeMax(4)
# pelvis_coordinates.setRangeMin(-4)

## Save the model

In [32]:
scaled.printToXML(output_model_file)

True

## Scale muscle fibres

In [13]:
# from fibre_scale_script import *
# # define models
# generic_model = '../templates/RajagopalModified_generic_copy.osim'
# model_to_update = output_model_file
# updated_model = output_model_file

# # run scripts
# osimModel_opt, SimInfo = optimMuscleParams(generic_model, model_to_update, 2, '3a_osim_markers/logging')

# # printing the optimized model
# osimModel_opt.printToXML(updated_model)

# Add joint center markers and save another model

In [33]:
model = osim.Model(output_model_file)
for body in model.getBodySet():
    if body.getName() == 'torso':
        torso_body = body

    if body.getName() == 'pelvis':
        pelvis_body = body
    if body.getName() == 'femur_r':
        femur_r_body = body
    if body.getName() == 'femur_l':
        femur_l_body = body

    if body.getName() == 'patella_r':
        patella_r_body = body
    if body.getName() == 'patella_l':
        patella_l_body = body

    if body.getName() == 'tibia_r':
        tibia_r_body = body
    if body.getName() == 'tibia_l':
        tibia_l_body = body

    if body.getName() == 'talus_r':
        talus_r_body = body
    if body.getName() == 'talus_l':
        talus_l_body = body

In [34]:
torso_m = osim.Marker("torso_origin_in_pelvis", torso_body, osim.Vec3(0,0,0))
femur_l_m = osim.Marker("femur_l_center_in_pelvis", femur_l_body, osim.Vec3(0,0,0))
femur_r_m = osim.Marker("femur_r_center_in_pelvis", femur_r_body, osim.Vec3(0,0,0))
patella_l_m = osim.Marker("patella_l_in_femur_l", patella_l_body, osim.Vec3(0,0,0))
patella_r_m = osim.Marker("patella_r_in_femur_r", patella_r_body, osim.Vec3(0,0,0))
tibia_l_m = osim.Marker("knee_l_center_in_femur_l", tibia_l_body, osim.Vec3(0,0,0))
tibia_r_m = osim.Marker("knee_r_center_in_femur_r", tibia_r_body, osim.Vec3(0,0,0))
talus_l_m = osim.Marker("talus_l_center_in_tibia", talus_l_body, osim.Vec3(0,0,0))
talus_r_m = osim.Marker("talus_r_center_in_tibia", talus_r_body, osim.Vec3(0,0,0))

model.addMarker(torso_m)
model.addMarker(femur_l_m)
model.addMarker(femur_r_m)
model.addMarker(patella_l_m)
model.addMarker(patella_r_m)
model.addMarker(tibia_l_m)
model.addMarker(tibia_r_m)
model.addMarker(talus_l_m)
model.addMarker(talus_r_m)

model.finalizeConnections()

In [35]:
state = model.initSystem()

torso_m.changeFramePreserveLocation(state, pelvis_body)
femur_l_m.changeFramePreserveLocation(state, pelvis_body)
femur_r_m.changeFramePreserveLocation(state, pelvis_body)
patella_l_m.changeFramePreserveLocation(state, femur_l_body)
patella_r_m.changeFramePreserveLocation(state, femur_r_body)
tibia_l_m.changeFramePreserveLocation(state, femur_l_body)
tibia_r_m.changeFramePreserveLocation(state, femur_r_body)
talus_l_m.changeFramePreserveLocation(state, tibia_l_body)
talus_r_m.changeFramePreserveLocation(state, tibia_r_body)

In [37]:
model.finalizeConnections()
model.printToXML(os.path.join(path_to_model, f'scaled_model_joints.osim'))

True